# Simple Transformer

In [ ]:
import time
import torch
import torch.nn as nn
import numpy as np

from torch.nn import functional as F

In [ ]:
torch.manual_seed(1337)

## Constants

In [ ]:
batch_size = 60
block_size = 512
max_iters = 5000
learning_rate = 2e-4
loss_est_n_batch = 2

embedding_size = 768
num_heads = 6
head_size = embedding_size
n_layer = 6
dropout = 0.3

## Hardware

In [ ]:
has_cuda = torch.cuda.is_available() and torch.backends.cuda.is_built()
has_mps = torch.backends.mps.is_available() and torch.backends.mps.is_built()
print(f'CUDA available: {has_cuda}')
if has_cuda:
    print(f'  * GPU count: {torch.cuda.device_count()}')
print(f'mps available: {has_mps}')

In [ ]:
device = "cpu"
if has_cuda:
    device = "cuda"
elif has_mps:
    device = "mps"
# device = "cpu"
torch.set_default_device(device)

## Training Data

In [ ]:
with open('data/tiny_shakespeare.txt', 'r', encoding='utf-8') as input:
    text = input.read()

In [ ]:
print(f'Number of characters in text: {len(text)}')

In [ ]:
print(f'First 100 chars:\n\n```\n{text[:100]}\n```')

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
vocab_size

In [ ]:
''.join(chars)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

In [ ]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
encoded_test = encode("hello world!")
encoded_test

In [ ]:
decode(encoded_test)

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
train_data[:block_size + 1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

## Generate Batch Data

In [ ]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb[:5])
print('targets:')
print(yb.shape)
print(yb[:5])

In [ ]:
for t in range(8):
    context = xb[0, :t+1]
    target = yb[0, t]
    print(f'input: {context}, target: {target}')

## Neural Net

In [ ]:
@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    def __init__(self, n_embd: int, head_size: int):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Input: (B, T, C)
        Ouput: (B, T, head_size)
        """
        B, T, C = x.shape

        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)

        wei = q @ k.transpose(-2, -1) * (C**-0.5) # (B, T, head_size) @ (B, head_size, T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        
        out = wei @ v # (B, T, T) @ (B, T, head_size) = (B, T, head_size)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd: int, n_heads: int, head_size: int):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head_size//n_heads) for _ in range(n_heads)])
        self.proj = nn.Linear(head_size, head_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Input: (B, T, C)
        Ouput: (B, T, head_size)
        """
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, head_size: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(head_size, head_size * 4),
            nn.ReLU(),
            nn.Linear(head_size * 4, head_size),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        """
        Input: (B, T, head_size)
        Ouput: (B, T, head_size)
        """
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, n_embd: int, n_head: int, head_size: int):
        super().__init__()
        self.sa = MultiHeadAttention(n_embd, n_head, head_size)
        self.ffwd = FeedForward(head_size)
        self.ln1 = nn.LayerNorm(head_size)
        self.ln2 = nn.LayerNorm(head_size)

    def forward(self, x):
        """
        Input: (B, T, C)
        Ouput: (B, T, head_size)
        """
        x = self.ln1(x)
        x = x + self.sa(x) # (B, T, C) ---> (B, T, head_size)
        x = self.ln2(x)
        x = x + self.ffwd(x) # (B, T, head_size) ---> (B, T, head_size)
        return x

In [ ]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int, n_embd: int, n_head: int, head_size: int, n_layer: int) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, head_size) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(head_size)
        self.lm_head = nn.Linear(head_size, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, C) batch, time, channel
        pos_emb = self.pos_embedding_table(torch.arange(T)) # (T, C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits.view(B*T, C), targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_conx = idx[:, -block_size:]
            logits, _ = self(idx_conx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
m = BigramLanguageModel(vocab_size, embedding_size, num_heads, head_size, n_layer)

In [ ]:
num_params = sum(p.numel() for p in m.parameters())
print("{num_params:.2f}M parameters".format(num_params=num_params/1e6))

In [ ]:
logits, loss = m(xb, yb)
print(f"logits shape: {logits.shape}")
print(f"loss: {loss}")

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long)
generated = m.generate(idx, max_new_tokens=100)
print(decode(generated[0].tolist()))

In [ ]:
estimate_loss(m, loss_est_n_batch)

## Training

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
train_step = 0

In [ ]:
start = time.time()
for _ in range(max_iters):
    train_step += 1
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if train_step <= 1 or train_step % (max_iters // 20) == 0:
        batch_loss = estimate_loss(m, loss_est_n_batch)
        print("Step {step}: train_loss={train:.5f}, val_loss={val:.5f}, dur={dur:.2f}s".format(step=train_step, train=batch_loss['train'], val=batch_loss['val'], dur=time.time() - start))

In [ ]:
start_text = "Sky, "
idx = torch.tensor(np.array(encode(start_text)), dtype=torch.long).view(1, len(start_text))
m.eval()
generated = m.generate(idx, max_new_tokens=300)
m.train()
print(decode(generated[0].tolist()))

## Self-Attention

In [ ]:
B, T, C = 11, 5, 7
x = torch.randn(B, T, C)

In [ ]:
head_size = 3
sqrt_dk = head_size ** -0.5

key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
v = value(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) * sqrt_dk # (B, T, head_size) @ (B, head_size, T) = (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

In [ ]:
out = wei @ v

In [ ]:
print(f"(B, T, C)=({B}, {T}, {C})")
print(f"x.shape={x.shape}")
print(f"k.shape={k.shape}")
print(f"q.shape={q.shape}")
print(f"v.shape={v.shape}")
print(f"wei.shape={wei.shape}")
print(f"out.shape={out.shape}")

In [ ]:
x[0]

In [ ]:
wei[0]

In [ ]:
out[0]